In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv(r'../data/sales.csv', index_col=0)

In [3]:
import sklearn
import hts

In [4]:
data.head()

,Product_Code,Warehouse,Product_Category,Date,Order_Demand,month_year,year
0,Product_0993,Whse_J,Category_028,2012-07-27,100.0,2012-07,2012
1,Product_0979,Whse_J,Category_028,2012-01-19,500.0,2012-01,2012
2,Product_0979,Whse_J,Category_028,2012-02-03,500.0,2012-02,2012
3,Product_0979,Whse_J,Category_028,2012-02-09,500.0,2012-02,2012
4,Product_0979,Whse_J,Category_028,2012-03-02,500.0,2012-03,2012


In [5]:
data["Warehouse"].unique()

array(['Whse_J', 'Whse_S', 'Whse_C', 'Whse_A'], dtype=object)

In [6]:
# abbreviating warehouse and product category
mapping_warehouse ={
    "Whse_J" : "WJ",
    "Whse_S" : "WS",
    "Whse_C": "WC",
    "Whse_A": "WA",
    
}
data["Warehouse"] = data["Warehouse"].map(mapping_warehouse)

In [7]:
data["Product_Category"].unique()

array(['Category_028', 'Category_006', 'Category_001', 'Category_019',
       'Category_005', 'Category_030', 'Category_032', 'Category_011',
       'Category_033', 'Category_007', 'Category_015', 'Category_021',
       'Category_024', 'Category_026', 'Category_023', 'Category_022',
       'Category_009', 'Category_003', 'Category_004', 'Category_018',
       'Category_020', 'Category_013', 'Category_008', 'Category_017',
       'Category_031', 'Category_025', 'Category_010', 'Category_029',
       'Category_012', 'Category_002', 'Category_014', 'Category_027',
       'Category_016'], dtype=object)

In [8]:
mapping_cat = {
    
}
categories = data["Product_Category"].unique()
for i in categories:
    abbre = i[:3] + i.split("_", 1)[1]
    mapping_cat[i] = abbre

In [9]:
mapping_cat

{'Category_028': 'Cat028',
 'Category_006': 'Cat006',
 'Category_001': 'Cat001',
 'Category_019': 'Cat019',
 'Category_005': 'Cat005',
 'Category_030': 'Cat030',
 'Category_032': 'Cat032',
 'Category_011': 'Cat011',
 'Category_033': 'Cat033',
 'Category_007': 'Cat007',
 'Category_015': 'Cat015',
 'Category_021': 'Cat021',
 'Category_024': 'Cat024',
 'Category_026': 'Cat026',
 'Category_023': 'Cat023',
 'Category_022': 'Cat022',
 'Category_009': 'Cat009',
 'Category_003': 'Cat003',
 'Category_004': 'Cat004',
 'Category_018': 'Cat018',
 'Category_020': 'Cat020',
 'Category_013': 'Cat013',
 'Category_008': 'Cat008',
 'Category_017': 'Cat017',
 'Category_031': 'Cat031',
 'Category_025': 'Cat025',
 'Category_010': 'Cat010',
 'Category_029': 'Cat029',
 'Category_012': 'Cat012',
 'Category_002': 'Cat002',
 'Category_014': 'Cat014',
 'Category_027': 'Cat027',
 'Category_016': 'Cat016'}

In [10]:
data["Product_Category"] = data["Product_Category"].map(mapping_cat)

In [11]:
data["warehouse_cat_id"] = data.apply(lambda x: f"{x['Warehouse']}_{x['Product_Category']}", axis=1)
data["prod_cat_id"] = data.apply(lambda x: f"{x['Product_Category']}_{x['Product_Code']}", axis=1)

In [12]:
data["warehouse_products"] = data.apply(lambda x:f"{x['warehouse_cat_id']}_{x['Product_Code']}", axis=1)

In [13]:
warehouses = data["Warehouse"].unique()
prod_cats = data["Product_Category"].unique()
cats = data["warehouse_cat_id"].unique()
products = data["prod_cat_id"].unique()
warehouse_prods = data["warehouse_products"].unique()

In [14]:
# build hierarchy
total = {'total': list(np.concatenate((warehouses, prod_cats)))}
warehouse = {i: [j for j in cats if j.startswith(i)] for i in warehouses}
category = {i: [j for j in products if j.startswith(i)] for i in prod_cats}
warehouse_prod = {i: [j for j in warehouse_prods if j.startswith(i)] for i in cats}
#product = {i: [j for j in products]}
hierarchy = {**total, **warehouse,**category, **warehouse_prod}

In [15]:
hierarchy

{'total': ['WJ',
  'WS',
  'WC',
  'WA',
  'Cat028',
  'Cat006',
  'Cat001',
  'Cat019',
  'Cat005',
  'Cat030',
  'Cat032',
  'Cat011',
  'Cat033',
  'Cat007',
  'Cat015',
  'Cat021',
  'Cat024',
  'Cat026',
  'Cat023',
  'Cat022',
  'Cat009',
  'Cat003',
  'Cat004',
  'Cat018',
  'Cat020',
  'Cat013',
  'Cat008',
  'Cat017',
  'Cat031',
  'Cat025',
  'Cat010',
  'Cat029',
  'Cat012',
  'Cat002',
  'Cat014',
  'Cat027',
  'Cat016'],
 'WJ': ['WJ_Cat028',
  'WJ_Cat006',
  'WJ_Cat001',
  'WJ_Cat019',
  'WJ_Cat005',
  'WJ_Cat011',
  'WJ_Cat033',
  'WJ_Cat007',
  'WJ_Cat015',
  'WJ_Cat021',
  'WJ_Cat024',
  'WJ_Cat026',
  'WJ_Cat013',
  'WJ_Cat009',
  'WJ_Cat022',
  'WJ_Cat003',
  'WJ_Cat008',
  'WJ_Cat030',
  'WJ_Cat017',
  'WJ_Cat031',
  'WJ_Cat032',
  'WJ_Cat023',
  'WJ_Cat018',
  'WJ_Cat010',
  'WJ_Cat012'],
 'WS': ['WS_Cat019',
  'WS_Cat030',
  'WS_Cat006',
  'WS_Cat028',
  'WS_Cat001',
  'WS_Cat015',
  'WS_Cat022',
  'WS_Cat007',
  'WS_Cat009',
  'WS_Cat021',
  'WS_Cat005',
  'WS_Cat

In [16]:
hierarchy_data = data.groupby(["month_year", "Warehouse", "Product_Category", "warehouse_cat_id", 
                               "prod_cat_id", "warehouse_products"]).sum("Order_Demand")
    

In [17]:
hierarchy_data = hierarchy_data.drop("year", axis=1)

In [18]:
#level1 = hierarchy.groupby()

In [19]:
hierarchy_data.head()

Order_Demand
month_year Warehouse Product_Category warehouse_cat_id prod_cat_id         warehouse_products                  
2011-01    WA        Cat006           WA_Cat006        Cat006_Product_0965 WA_Cat006_Product_0965           2.0
2011-05    WA        Cat003           WA_Cat003        Cat003_Product_1724 WA_Cat003_Product_1724         108.0
2011-06    WS        Cat019           WS_Cat019        Cat019_Product_1521 WS_Cat019_Product_1521       92000.0
2011-09    WC        Cat001           WC_Cat001        Cat001_Product_0608 WC_Cat001_Product_0608           5.0
                                                       Cat001_Product_1933 WC_Cat001_Product_1933          23.0

In [20]:
hierarchy_data = hierarchy_data.reset_index()

In [21]:
#bottom level dataframe 
bottom = hierarchy_data.pivot(index = "month_year", columns = "warehouse_products", values = "Order_Demand")

In [22]:
bottom.head()

warehouse_products,WA_Cat001_Product_1751,WA_Cat003_Product_1180,WA_Cat003_Product_1719,WA_Cat003_Product_1724,WA_Cat003_Product_1725,WA_Cat003_Product_1726,WA_Cat003_Product_1727,WA_Cat005_Product_0001,WA_Cat005_Product_0020,WA_Cat005_Product_0031,...,WS_Cat032_Product_0586,WS_Cat032_Product_0587,WS_Cat032_Product_0588,WS_Cat032_Product_0589,WS_Cat032_Product_0591,WS_Cat032_Product_0592,WS_Cat032_Product_0593,WS_Cat032_Product_0595,WS_Cat032_Product_1052,WS_Cat032_Product_1101
month_year,,,,,,,,,,,,,,,,,,,,,
2011-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-05,NaN,NaN,NaN,108.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
mid_level1 = hierarchy_data.groupby(["month_year", "prod_cat_id"]).sum().\
    reset_index(drop = False).\
        pivot(index= "month_year", columns = "prod_cat_id", values = "Order_Demand")

C:\Users\HP\AppData\Local\Temp\ipykernel_11496\944314362.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mid_level1 = hierarchy_data.groupby(["month_year", "prod_cat_id"]).sum().\


In [24]:
mid_level2 = hierarchy_data.groupby(["month_year", "warehouse_cat_id"]).sum().\
    reset_index(drop = False).\
        pivot(index= "month_year", columns = "warehouse_cat_id", values = "Order_Demand")

C:\Users\HP\AppData\Local\Temp\ipykernel_11496\693054579.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mid_level2 = hierarchy_data.groupby(["month_year", "warehouse_cat_id"]).sum().\


In [25]:
mid_level3 = hierarchy_data.groupby(["month_year", "Product_Category"]).sum().\
    reset_index(drop = False).\
        pivot(index= "month_year", columns = "Product_Category", values = "Order_Demand")

C:\Users\HP\AppData\Local\Temp\ipykernel_11496\642283387.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mid_level3 = hierarchy_data.groupby(["month_year", "Product_Category"]).sum().\


In [26]:
mid_level4 = hierarchy_data.groupby(["month_year", "Warehouse"]).sum().\
    reset_index(drop = False).\
        pivot(index= "month_year", columns = "Warehouse", values = "Order_Demand")

C:\Users\HP\AppData\Local\Temp\ipykernel_11496\1092086419.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mid_level4 = hierarchy_data.groupby(["month_year", "Warehouse"]).sum().\


In [29]:
top_level = hierarchy_data.groupby("month_year").sum().rename(columns = {"Order_Demand":"total"})

C:\Users\HP\AppData\Local\Temp\ipykernel_11496\3608796570.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  top_level = hierarchy_data.groupby("month_year").sum().rename(columns = {"Order_Demand":"total"})


In [30]:
total_df = bottom.join(mid_level1).join(mid_level2).join(mid_level3).join(mid_level4).join(top_level)

In [31]:
total_df.index = pd.to_datetime(total_df.index)

In [32]:
total_df = total_df.resample("MS").sum()

In [33]:
total_df.head()

,WA_Cat001_Product_1751,WA_Cat003_Product_1180,WA_Cat003_Product_1719,WA_Cat003_Product_1724,WA_Cat003_Product_1725,WA_Cat003_Product_1726,WA_Cat003_Product_1727,WA_Cat005_Product_0001,WA_Cat005_Product_0020,WA_Cat005_Product_0031,...,Cat029,Cat030,Cat031,Cat032,Cat033,WA,WC,WJ,WS,total
month_year,,,,,,,,,,,,,,,,,,,,,
2011-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0
2011-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-04-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-05-01,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,108.0


In [34]:
total_df.columns

Index(['WA_Cat001_Product_1751', 'WA_Cat003_Product_1180',
       'WA_Cat003_Product_1719', 'WA_Cat003_Product_1724',
       'WA_Cat003_Product_1725', 'WA_Cat003_Product_1726',
       'WA_Cat003_Product_1727', 'WA_Cat005_Product_0001',
       'WA_Cat005_Product_0020', 'WA_Cat005_Product_0031',
       ...
       'Cat029', 'Cat030', 'Cat031', 'Cat032', 'Cat033', 'WA', 'WC', 'WJ',
       'WS', 'total'],
      dtype='object', length=5141)

In [35]:
total_df.index

DatetimeIndex(['2011-01-01', '2011-02-01', '2011-03-01', '2011-04-01',
               '2011-05-01', '2011-06-01', '2011-07-01', '2011-08-01',
               '2011-09-01', '2011-10-01', '2011-11-01', '2011-12-01',
               '2012-01-01', '2012-02-01', '2012-03-01', '2012-04-01',
               '2012-05-01', '2012-06-01', '2012-07-01', '2012-08-01',
               '2012-09-01', '2012-10-01', '2012-11-01', '2012-12-01',
               '2013-01-01', '2013-02-01', '2013-03-01', '2013-04-01',
               '2013-05-01', '2013-06-01', '2013-07-01', '2013-08-01',
               '2013-09-01', '2013-10-01', '2013-11-01', '2013-12-01',
               '2014-01-01', '2014-02-01', '2014-03-01', '2014-04-01',
               '2014-05-01', '2014-06-01', '2014-07-01', '2014-08-01',
               '2014-09-01', '2014-10-01', '2014-11-01', '2014-12-01',
               '2015-01-01', '2015-02-01', '2015-03-01', '2015-04-01',
               '2015-05-01', '2015-06-01', '2015-07-01', '2015-08-01',
      

In [36]:
from hts.hierarchy import HierarchyTree

In [37]:
tree = HierarchyTree.from_nodes(hierarchy, total_df)

# HTS FORECASTING WITH ARIMA

In [52]:
import pmdarima as pm
from pmdarima import auto_arima

In [53]:
train = total_df["2012-01-01":"2015-01-01"]
validation = total_df["2015-02-01" :]

In [54]:
train = train.fillna(0)

In [57]:
model_arima = hts.HTSRegressor(model = 'auto_arima', revision_method ='BU', n_jobs = 0)

In [58]:
model_arima = model_arima.fit(train, hierarchy)

Fitting models: 100%|██████████| 5141/5141 [54:55<00:00,  1.56it/s]  


In [59]:
train.tail()

,WA_Cat001_Product_1751,WA_Cat003_Product_1180,WA_Cat003_Product_1719,WA_Cat003_Product_1724,WA_Cat003_Product_1725,WA_Cat003_Product_1726,WA_Cat003_Product_1727,WA_Cat005_Product_0001,WA_Cat005_Product_0020,WA_Cat005_Product_0031,...,Cat029,Cat030,Cat031,Cat032,Cat033,WA,WC,WJ,WS,total
month_year,,,,,,,,,,,,,,,,,,,,,
2014-09-01,121.0,72.0,432.0,459.0,0.0,0.0,36.0,6600.0,5000.0,12800.0,...,366.0,702800.0,186.0,94290.0,650000.0,2337400.0,9275619.0,59749262.0,18072843.0,89435124.0
2014-10-01,100.0,54.0,513.0,1755.0,0.0,0.0,108.0,10200.0,2000.0,2400.0,...,150.0,937350.0,77.0,84334.0,590000.0,2546151.0,9155745.0,68379118.0,17088092.0,97169106.0
2014-11-01,60.0,162.0,351.0,999.0,0.0,0.0,72.0,8800.0,1500.0,4800.0,...,130.0,896202.0,251.0,106356.0,650000.0,2471183.0,8851897.0,57028852.0,21037287.0,89389219.0
2014-12-01,0.0,36.0,81.0,1350.0,0.0,0.0,54.0,4000.0,200.0,20500.0,...,412.0,898701.0,83.0,227414.0,630000.0,2012032.0,9800071.0,60342181.0,21697892.0,93852176.0
2015-01-01,0.0,198.0,243.0,486.0,0.0,0.0,54.0,2100.0,1700.0,1900.0,...,213.0,1139228.0,142.0,62268.0,680000.0,2198807.0,12849928.0,63914873.0,24871498.0,103835106.0


In [60]:
prediction_bu = model_arima.predict(steps_ahead=12)

Fitting models: 100%|██████████| 5141/5141 [00:34<00:00, 149.69it/s]


In [66]:
prediction = prediction_bu['2015-02-01':]

In [67]:
prediction

,total,WJ,WS,WC,WA,Cat028,Cat006,Cat001,Cat019,Cat005,...,WA_Cat012_Product_0567,WA_Cat012_Product_0437,WA_Cat012_Product_0466,WA_Cat012_Product_0465,WA_Cat008_Product_0149,WA_Cat008_Product_0119,WA_Cat001_Product_1751,WA_Cat027_Product_2147,WA_Cat027_Product_0647,WA_Cat016_Product_1083
2015-02-01,9.245362e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.864865,17.766276,0.0,...,0.0,0.0,0.0,0.0,16.324324,9.204176,0.0,0.0,0.0,0.0
2015-03-01,9.210854e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.864865,17.766276,0.0,...,0.0,0.0,0.0,0.0,16.324324,9.464075,0.0,0.0,0.0,0.0
2015-04-01,9.189412e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.864865,17.766276,0.0,...,0.0,0.0,0.0,0.0,16.324324,9.379198,0.0,0.0,0.0,0.0
2015-05-01,9.234595e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.864865,17.766276,0.0,...,0.0,0.0,0.0,0.0,16.324324,9.406917,0.0,0.0,0.0,0.0
2015-06-01,9.191178e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.864865,17.766276,0.0,...,0.0,0.0,0.0,0.0,16.324324,9.397864,0.0,0.0,0.0,0.0
2015-07-01,9.235886e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.864865,17.766276,0.0,...,0.0,0.0,0.0,0.0,16.324324,9.400821,0.0,0.0,0.0,0.0
2015-08-01,9.242723e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.864865,17.766276,0.0,...,0.0,0.0,0.0,0.0,16.324324,9.399855,0.0,0.0,0.0,0.0
2015-09-01,9.263382e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.864865,17.766276,0.0,...,0.0,0.0,0.0,0.0,16.324324,9.400170,0.0,0.0,0.0,0.0
2015-10-01,9.259478e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.864865,17.766276,0.0,...,0.0,0.0,0.0,0.0,16.324324,9.400067,0.0,0.0,0.0,0.0
2015-11-01,9.297554e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.864865,17.766276,0.0,...,0.0,0.0,0.0,0.0,16.324324,9.400101,0.0,0.0,0.0,0.0


In [68]:
from sklearn.metrics import mean_squared_error

In [70]:
# model accuracy
cols = total_df.columns

for i in cols:
    print('rmse value for', i, 'is : ', np.sqrt(mean_squared_error(prediction[i], validation[:'2016-01-01'][i])))

rmse value for WA_Cat001_Product_1751 is :  16.232683080747925
rmse value for WA_Cat003_Product_1180 is :  236.57740800441235
rmse value for WA_Cat003_Product_1719 is :  166.39158734140946
rmse value for WA_Cat003_Product_1724 is :  966.5472052621122
rmse value for WA_Cat003_Product_1725 is :  23.811761799581316
rmse value for WA_Cat003_Product_1726 is :  842.5621638787253
rmse value for WA_Cat003_Product_1727 is :  53.05320292571149
rmse value for WA_Cat005_Product_0001 is :  3628.150377858331
rmse value for WA_Cat005_Product_0020 is :  2259.892222230819
rmse value for WA_Cat005_Product_0031 is :  2653.2233200161977
rmse value for WA_Cat005_Product_1022 is :  1401.2770941021106
rmse value for WA_Cat006_Product_0935 is :  4461.8202869126335
rmse value for WA_Cat006_Product_0936 is :  39712.72193654937
rmse value for WA_Cat006_Product_0937 is :  19935.610285641902
rmse value for WA_Cat006_Product_0965 is :  5.082829869650188
rmse value for WA_Cat006_Product_0966 is :  32.547842705277255

In [71]:
from sklearn.metrics import r2_score

In [76]:
r2_arima = r2_score(prediction.to_numpy(), validation[:'2016-01-01'].to_numpy())

In [77]:
r2

-2.9011592557815777e+194